#### Implementing LangGraph's persistence layer for fault tolerance and facilitate conversations (remember previous interactions)

In [1]:
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
import os 

load_dotenv()

# GPT_MODEL = 'gpt-4o'
model = ChatOpenAI(model="gpt-4o")

In [2]:
# API Configuration
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import START, MessagesState, StateGraph
from psycopg import Connection

In [4]:
# Database connection details
DB_URI = os.getenv("DB_URI")

connection_kwargs = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": "5432",
    "autocommit": True,
    "prepare_threshold": 0,
}

In [5]:
# Define LangGraph workflow
workflow = StateGraph(state_schema=MessagesState)

In [6]:
# Define function to call the model

def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}

In [7]:
# Add the node and edge to the workflow

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

In [55]:
with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # first time using database/table only
    # checkpointer.setup()

    # compile the workflow with the Postgres checkpointer
    app = workflow.compile(checkpointer=checkpointer)

    # Example configuration and input
    config = {"configurable": {"thread_id": "1"}}
    input_data = {"messages": [("human", "what's the weather in Denver?")]}

    # run the workflow
    response = app.invoke(input_data, config)

    # retrieve the checkpoint tuple
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("Workflow Response:", response)
    print("Checkpoint Tuple:", checkpoint_tuple)

Workflow Response: {'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'), AIMessage(content="I'm unable to provide real-time weather updates. To get the current weather in Denver, I recommend checking a reliable weather website or app like the Weather Channel, the National Weather Service, or a weather app on your smartphone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a692fdca-00cd-4ed7-8075-6fff37822903-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, '

In [56]:
checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efdbd30-0d9e-67bc-8004-b7cff6ffd96d'}}, checkpoint={'v': 1, 'id': '1efdbd30-0d9e-67bc-8004-b7cff6ffd96d', 'ts': '2025-01-26T10:47:57.063775+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000005.0.7492994878569476'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000004.0.24078976381853234'}}, 'channel_versions': {'model': '00000000000000000000000000000006.0.9491617378314615', 'messages': '00000000000000000000000000000006.0.08219908600831227', '__start__': '00000000000000000000000000000005.0.11068437511918705', 'start:model': '00000000000000000000000000000006.0.6516098280634158'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'), AIMessage(content="I'm unable to provide real-

In [66]:
# Test persistence

with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # first time using database/table only
    # checkpointer.setup()

    # compile the workflow with the Postgres checkpointer
    app = workflow.compile(checkpointer=checkpointer)

    config = {"configurable": {"thread_id": "1"}}
    input_data = {"messages": [("human", "What city did I ask you for weather information?")]}

    response = app.invoke(input_data, config)

    checkpoint_tuple = checkpointer.get_tuple(config)

    print("Workflow Response:", response)
    print("Checkpoint Tuple:", checkpoint_tuple)

Workflow Response: {'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'), AIMessage(content="I'm unable to provide real-time weather updates. To get the current weather in Denver, I recommend checking a reliable weather website or app like the Weather Channel, the National Weather Service, or a weather app on your smartphone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a692fdca-00cd-4ed7-8075-6fff37822903-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, '

In [67]:
response

{'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'),
  AIMessage(content="I'm unable to provide real-time weather updates. To get the current weather in Denver, I recommend checking a reliable weather website or app like the Weather Channel, the National Weather Service, or a weather app on your smartphone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a692fdca-00cd-4ed7-8075-6fff37822903-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, 'total_tokens': 79

In [12]:
with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # Compile workflow with persistence
    app = workflow.compile(checkpointer=checkpointer)

    # user_prompt = "how can I save energy use at home?"

    # specify configuration and input
    config = {"configurable": {"thread_id": "44"}}
    # input_data = {"messages": [("human", test_prompt)]}

    # run the workflow
    # response = app.invoke(input_data, config)
    # print("Workflow response:", response)

    # retrieve the checkpoint tuple
    checkpoint_tuple = checkpointer.get_tuple(config)

    print(checkpoint_tuple)

CheckpointTuple(config={'configurable': {'thread_id': '44', 'checkpoint_ns': '', 'checkpoint_id': '1effd582-0825-6b3b-800a-e3e411ad1f59'}}, checkpoint={'v': 1, 'id': '1effd582-0825-6b3b-800a-e3e411ad1f59', 'ts': '2025-03-10T02:34:01.613804+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000011.0.5497762592228551'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000010.0.9553892184761255'}}, 'channel_versions': {'model': '00000000000000000000000000000012.0.3387068263476115', 'messages': '00000000000000000000000000000012.0.30487647493590764', '__start__': '00000000000000000000000000000011.0.6084132299793199', 'start:model': '00000000000000000000000000000012.0.29189869019904746'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='80157cea-ad31-494d-92e1-a3e655160f01'), HumanMessage(con

CheckpointTuple(config={'configurable': {'thread_id': '44', 'checkpoint_ns': '', 'checkpoint_id': '1effd563-c0f6-660e-8007-b1fd02536226'}}, checkpoint={'v': 1, 'id': '1effd563-c0f6-660e-8007-b1fd02536226', 'ts': '2025-03-10T02:20:28.843163+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000008.0.7879228690242339'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000007.0.9539969430167892'}}, 'channel_versions': {'model': '00000000000000000000000000000009.0.8027394438804663', 'messages': '00000000000000000000000000000009.0.364134698123025', '__start__': '00000000000000000000000000000008.0.1449590072310507', 'start:model': '00000000000000000000000000000009.0.7415295891672022'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='80157cea-ad31-494d-92e1-a3e655160f01'), HumanMessage(content='The part of the house that consumed the most energy in August was furnace_2 with a total energy consumption of 5484.28.', additional_kwargs={}, response_metadata={}, id='97d7bfd6-82b3-4659-939e-25c70ab495ab'), HumanMessage(content='Which part of the house consumed the most energy in September?', additional_kwargs={}, response_metadata={}, id='fcf62199-3a2e-4391-99f4-e9e982cc972c'), HumanMessage(content='furnace_2', additional_kwargs={}, response_metadata={}, id='f40674d0-4ff4-4c28-9c02-cece1ed03289'), HumanMessage(content='Which part of the house consumed the most energy in January?', additional_kwargs={}, response_metadata={}, id='9ac39618-ec79-43dd-aa88-0b34a02f024f'), HumanMessage(content='The part of the house that consumed the most energy in August was furnace_2.', additional_kwargs={}, response_metadata={}, id='8c0da367-68ef-4355-b589-65544b8c56ab')]}}, metadata={'step': 7, 'source': 'loop', 'writes': {'model': {'messages': 'The part of the house that consumed the most energy in August was furnace_2.'}}, 'parents': {}, 'thread_id': '44'}, parent_config={'configurable': {'thread_id': '44', 'checkpoint_ns': '', 'checkpoint_id': '1effd563-a34d-61cf-8006-292768f62f01'}}, pending_writes=[])

CheckpointTuple(config={'configurable': {'thread_id': '55', 'checkpoint_ns': '', 'checkpoint_id': '1effd548-5c2f-67c8-807f-0e6a2faf3f2b'}}, checkpoint={'v': 1, 'id': '1effd548-5c2f-67c8-807f-0e6a2faf3f2b', 'ts': '2025-03-10T02:08:13.500205+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000128.0.07778205841693142'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000127.0.9825552500977863'}}, 'channel_versions': {'model': '00000000000000000000000000000129.0.2510549279673435', 'messages': '00000000000000000000000000000129.0.9316608244873016', '__start__': '00000000000000000000000000000128.0.10386178226428089', 'start:model': '00000000000000000000000000000129.0.28821668837396575'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content='which room in my house used the most energy in July?', additional_kwargs={}, response_metadata={}, id='f03ba896-acdb-4805-a6e3-baa4f9c727b0'), HumanMessage(content='Living Room', additional_kwargs={}, response_metadata={}, id='48d3e125-50c1-46e2-95e9-83b554e93a2e'), HumanMessage(content='what did I just ask you about??', additional_kwargs={}, response_metadata={}, id='6b3eeccf-a494-468b-a7e3-18202ad360a5'), AIMessage(content='You asked about which room in your house used the most energy in July.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 53, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aff5de81-d3f8-4df1-b6d9-32a007f1cc9c-0'), HumanMessage(content='what did I just ask you about?', additional_kwargs={}, response_metadata={}, id='cdb64b72-c11e-45d7-aabc-a7eb920df446'), AIMessage(content='You just asked me about which room in your house used the most energy in July.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 84, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1f952960-2618-40a5-8207-360e692f6342-0'), HumanMessage(content='what did I ask you about?', additional_kwargs={}, response_metadata={}, id='0e78d4b8-c424-46df-8a95-bb892ddde281'), AIMessage(content='You asked me about utility bills and how to understand them.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 94, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-48ad1a6c-a48b-4a4b-b012-10e1ccd94d23-0'), HumanMessage(content='what did I ask you?', additional_kwargs={}, response_metadata={}, id='25288374-7245-4c6f-849d-5395055c5563'), AIMessage(content='You asked me to help you understand utility bills.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 89, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01d0f871-e28c-4ed2-8b94-95b899bd48e5-0'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='6c9cd223-eaed-4f2c-940c-8949d1c85487'), HumanMessage(content='Agent stopped due to iteration limit or time limit.', additional_kwargs={}, response_metadata={}, id='9f9304ac-821f-49e6-893d-80f2926347a8'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='10e773ad-5409-46e3-9f7d-b047482cc94d'), HumanMessage(content='The part of the house that consumed the most energy in August was the living room.', additional_kwargs={}, response_metadata={}, id='d26fe90b-4d8b-4081-b611-64dbde78d02c'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='63ed9356-60b8-4ad9-b976-0f01e0750d21'), HumanMessage(content='The living room consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='7c72beb0-7574-4ec1-a82b-11495a99d1b0'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='a7095fd4-34df-4583-b0bb-10938a40abd1'), HumanMessage(content='The living room', additional_kwargs={}, response_metadata={}, id='dbb81040-19c9-487f-9c63-54e57cba7285'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='38faaccd-5985-4c83-ba92-183c251decc7'), HumanMessage(content='The living room consumed a total of 1753.47 units of energy in August.', additional_kwargs={}, response_metadata={}, id='83637585-bb16-4425-894f-1764c6c55944'), HumanMessage(content='what did I just ask you?', additional_kwargs={}, response_metadata={}, id='535f00d0-c807-4f04-b286-9bc26ac0e75c'), HumanMessage(content='The living room consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='cfc6a37d-a12a-49f0-8ed0-aea100f053b9'), HumanMessage(content='what did I last ask you?', additional_kwargs={}, response_metadata={}, id='ce78e167-d9a3-4569-8b66-9bfd9adf4287'), HumanMessage(content='The living room consumed a total of 1753.47 units of energy in August.', additional_kwargs={}, response_metadata={}, id='e7d6127e-9130-4124-adf5-9a74ae29ebe6'), HumanMessage(content='what did I just ask you?', additional_kwargs={}, response_metadata={}, id='bada42a0-e340-4771-8e50-922113645b9b'), HumanMessage(content='1753.47 units', additional_kwargs={}, response_metadata={}, id='1c7b9acb-1c95-475a-806d-9ad5280a3ba3'), HumanMessage(content='what did I last ask you?', additional_kwargs={}, response_metadata={}, id='2cd6f337-fb2b-4d31-83a1-be49c2c10c9c'), HumanMessage(content='Agent stopped due to iteration limit or time limit.', additional_kwargs={}, response_metadata={}, id='56ec54f0-95cb-4e9a-894d-e92213e8464e'), HumanMessage(content='what did I last ask you?', additional_kwargs={}, response_metadata={}, id='7c5e0499-8920-4165-95d0-87cca2f5b078'), HumanMessage(content='14.71456667 units', additional_kwargs={}, response_metadata={}, id='0423dc2a-2ac3-438c-bd77-b60fbcc5e4a3'), HumanMessage(content='what did I just ask you?', additional_kwargs={}, response_metadata={}, id='2f35975b-b4f2-492d-95b6-0c4e30fa7203'), HumanMessage(content='14.71456667 units', additional_kwargs={}, response_metadata={}, id='48e945e9-80b3-49f4-83e9-5f59cf219c13'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='5ca6177c-120a-47a2-8602-b066f1129a09'), HumanMessage(content='The part of the house that consumed the most energy in August was the kitchen_14, with a usage of 14.71456667 units.', additional_kwargs={}, response_metadata={}, id='112431a5-c81d-4f00-8208-56f868f7685c'), HumanMessage(content='what did I just ask you?', additional_kwargs={}, response_metadata={}, id='39f5e040-da2e-4767-ba30-d1b50e516532'), HumanMessage(content="I don't know.", additional_kwargs={}, response_metadata={}, id='5ba44abe-3e35-4963-ad12-f9b8e4172e2c'), HumanMessage(content='what did I last ask you?', additional_kwargs={}, response_metadata={}, id='449b7f41-9826-44c6-a285-a14877d21cde'), HumanMessage(content='The part of the house that consumed the most energy in August was the kitchen_14.', additional_kwargs={}, response_metadata={}, id='ed3a733b-a669-4a2d-9c03-4b83b2a50598'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='b23956aa-2894-4bbe-8929-1bd4549f4184'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='a12881ff-e1e2-4945-8479-47ad66217590'), HumanMessage(content='what did I just ask you?', additional_kwargs={}, response_metadata={}, id='8c8e99b6-42f9-4fb9-8d08-a70097a6184b'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='ee89ce23-c67e-4844-b9c4-4221d8c01637'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='3dd37122-6f11-40e3-94bb-d847150cc6c8'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='888a1b51-f726-471f-8994-ca97ced02429'), HumanMessage(content='what did I last as you?', additional_kwargs={}, response_metadata={}, id='08da573e-3dd4-422e-86be-53840494e9cc'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='7815287b-2e0f-42af-872d-9b9c10a164db'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='23cd11db-fb5d-49ed-a401-1603d35d7e2b'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='961dd57d-39fc-4a4c-a562-2bfca8d38977'), HumanMessage(content='what did I last ask you?', additional_kwargs={}, response_metadata={}, id='14d11c9a-02ce-4378-a494-13c56d6bb743'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='9f0b6680-dd38-46d9-85ab-ad632ec81bfd'), HumanMessage(content='Which room consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='e3aa2541-f296-4992-9aeb-34ef5643c546'), HumanMessage(content='The part of the house that consumed the most energy in August was kitchen_12.', additional_kwargs={}, response_metadata={}, id='f30f0e2c-7442-49b0-817e-75c03076c7b1'), HumanMessage(content='what did I just ask you about?', additional_kwargs={}, response_metadata={}, id='f16ec977-9466-4ac0-8e44-69325f74263d'), HumanMessage(content='The room that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='bf1ecf27-518f-4769-ba93-972633dbe527'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='b5785755-502a-4672-a3a5-324c933fb43a'), HumanMessage(content='The room that consumed the most energy in August was kitchen_12.', additional_kwargs={}, response_metadata={}, id='8e2dbf15-f911-4d0f-9971-c724409ec272'), HumanMessage(content='Which part of the house consumed the most energy in July?', additional_kwargs={}, response_metadata={}, id='0242c9e2-43aa-4dbd-b115-31bb268a2875'), HumanMessage(content='kitchen_12', additional_kwargs={}, response_metadata={}, id='4c9ab016-99de-4bdf-9397-008c1462451f'), HumanMessage(content='Which room in my house consumed the most energy in July?', additional_kwargs={}, response_metadata={}, id='10b5160c-81ee-4542-8d3b-2c0a25fe2a79'), HumanMessage(content='The room that consumed the most energy in August was kitchen_14.', additional_kwargs={}, response_metadata={}, id='0289f52d-f791-47c8-b60b-d8ac52f3e42c'), HumanMessage(content='Which part of the house consumed the most energy in July?', additional_kwargs={}, response_metadata={}, id='cd061900-5029-4d70-9100-6085f1536cde'), HumanMessage(content='kitchen_14', additional_kwargs={}, response_metadata={}, id='5b7a606d-6c36-46ff-a11c-7cdfb89c4422'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='65222e51-746a-43fe-955f-7a4ee4674499'), HumanMessage(content='kitchen_12', additional_kwargs={}, response_metadata={}, id='89c23659-3d37-4a92-af51-f7b401a732ef'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='59c11e0c-6449-4bb3-9451-8426bfd36f05'), HumanMessage(content='The kitchen_12 part of the house consumed the most energy in July, and the kitchen_14 part of the house consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='db8f4b88-4bc6-4d2b-b0d2-3f380dbbaa1f'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='1f02439f-8255-4ec7-9d61-392366276d24'), HumanMessage(content='The kitchen_12 part of the house consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='a1b6dce4-9148-451e-85f0-a145a36eccdf'), HumanMessage(content='Which part of the house consumed the most energy in February?', additional_kwargs={}, response_metadata={}, id='099bd955-8b71-42f4-bfcc-7c2d8757b08e'), HumanMessage(content='The kitchen_14 part of the house consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='14c2e867-206a-4c24-879e-d6f11432614b'), HumanMessage(content='Which part of the house consumed the most energy in March?', additional_kwargs={}, response_metadata={}, id='712af592-ade3-48c9-99ce-99dfbf9c79a3'), HumanMessage(content='The kitchen_12 part of the house consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='8a9c154b-d9b7-455d-98c4-f7f75104b1c7'), HumanMessage(content='which appliance used the most energy in January?', additional_kwargs={}, response_metadata={}, id='75b91911-cec7-4f4b-9be5-7d792b7cfe61'), HumanMessage(content='The kitchen_12 part of the house consumed the most energy in February.', additional_kwargs={}, response_metadata={}, id='f10bd18b-1130-4d8f-9b6f-78aeaec6bf2d'), HumanMessage(content='which room used the most energy in December?', additional_kwargs={}, response_metadata={}, id='bf5275ec-67ab-4a92-a6e2-d04dbddfdf92'), HumanMessage(content='The kitchen_12 part of the house consumed the most energy in March.', additional_kwargs={}, response_metadata={}, id='f3fca3ea-03a9-4b01-ae12-e489d32e4c72'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='38d2ddae-617d-445b-ae91-cc5a6ba154e9'), HumanMessage(content='dishwasher', additional_kwargs={}, response_metadata={}, id='b0ed600b-75f8-4d7e-82f0-efe535b9e0c5'), HumanMessage(content='which room used the most energy in December?', additional_kwargs={}, response_metadata={}, id='d42c6d09-6b1d-4ce9-8987-cc719fa653e0'), HumanMessage(content='The room that used the most energy in December was the dishwasher, with a value of 1.385266667.', additional_kwargs={}, response_metadata={}, id='d3bbb172-a730-48a3-8356-e648149da259'), HumanMessage(content='which room used the most energy in May?', additional_kwargs={}, response_metadata={}, id='f79a3707-df7b-4cdf-bc53-5fce93d653df'), HumanMessage(content='The dishwasher consumed the most energy in August.', additional_kwargs={}, response_metadata={}, id='1deb5d71-3405-4cb2-98d6-e49f66c80615'), HumanMessage(content='which room used the most energy in May?', additional_kwargs={}, response_metadata={}, id='1529fd14-eaba-4ce0-92b2-0eafadca1272'), HumanMessage(content='The room that used the most energy in May was the dishwasher, with a value of 1.378216667.', additional_kwargs={}, response_metadata={}, id='1d3f037c-484a-4f73-91f6-897d00973a69'), HumanMessage(content='Which part of the house consumed the most energy in August?', additional_kwargs={}, response_metadata={}, id='d68d0197-61ef-4b39-82a3-3ed1cdc7325c'), HumanMessage(content='The room that used the most energy in May was the dishwasher, with a value of 1.378216667.', additional_kwargs={}, response_metadata={}, id='e9a50709-c99d-488b-b5c4-1f3e88a949c4')]}}, metadata={'step': 127, 'source': 'loop', 'writes': {'model': {'messages': 'The room that used the most energy in May was the dishwasher, with a value of 1.378216667.'}}, 'parents': {}, 'thread_id': '55'}, parent_config={'configurable': {'thread_id': '55', 'checkpoint_ns': '', 'checkpoint_id': '1effd548-3a08-6c2a-807e-0e8e9c1a23f0'}}, pending_writes=[])

In [9]:
# import psycopg

# try:
#     # Establish the connection
#     with psycopg.connect(DB_URI, **connection_kwargs) as conn:
#         print("Connection successful!")

#         # Execute a test query
#         with conn.cursor() as cur:
#             cur.execute("""
#                 SELECT table_name
#                 FROM information_schema.tables
#                 WHERE table_schema = 'public'
#                 ORDER BY table_name;
#             """)
#             tables = cur.fetchall()
            
#             # Print the list of tables
#             print("Tables in the database:")
#             for table in tables:
#                 print(f"- {table[0]}")

# except Exception as e:
#     print("Error connecting to PostgreSQL:", e)